## RANDOM FOREST

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix
import matplotlib.image as pltimg

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn import metrics
from sklearn.metrics import r2_score

- Load Dataset

In [2]:
df = pd.read_csv('/home/forhad/Study/Self_Work/shazad/EDA/Stainless-Steel-Prices-Forecasty-Assignment.csv')

In [3]:
df

,Date,StainlessSteelPrice,Steel_Inventory_M_USD,Steel_Orders_M_USD,IronOre_Global_USD,CoalAustralia_Global_USD,Energy_Index,Nickel_Global_USD,Zinc_Global_USD,Freight_Transport_Index,WTI_Crude_Global_USD,Baltic_Dry_Index,BCI_China,BCI_Europe,BCI_US,CLI_China,CLI_Europe,CLI_US,PMI_US_Manufacturing,Copper_Global_USD
0,07-01-2013,0.827352,20575,10568,127.191304,82.222360,227.022942,13750.31522,1837.619565,-0.4,104.545217,1062,98.80018,99.64265,100.10440,101.14230,99.43615,100.62300,50.9,6906.64
1,08-01-2013,0.760221,20609,10821,137.055000,82.221429,232.234194,14308.26190,1896.392857,0.5,106.546364,1132,99.04874,99.91766,100.39620,101.19380,99.64082,100.65450,55.4,7186.25
2,09-01-2013,0.784715,20761,10774,134.185714,83.295918,234.335594,13801.39286,1846.880952,0.1,106.313810,2003,99.27196,100.18490,100.53630,101.20660,99.84383,100.66460,55.7,7159.27
3,10-01-2013,0.769293,20735,11291,132.572727,85.443634,229.393932,14117.65217,1884.836957,-0.1,100.500435,1504,99.40488,100.36330,100.57710,101.18350,100.02300,100.66660,56.2,7203.02
4,11-01-2013,0.750242,20854,11559,136.323810,88.363265,225.437049,13684.01190,1866.416667,2.1,93.812381,1821,99.36221,100.49830,100.54590,101.12830,100.17150,100.67440,56.4,7070.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,02-01-2020,0.893123,19753,9914,87.625000,71.050179,110.312331,12715.55000,2113.237500,-0.6,50.601500,535,94.50153,99.15099,98.99674,85.38329,99.33393,99.23477,50.9,5687.75
80,03-01-2020,0.829167,19945,9072,88.659091,70.204870,73.684533,11846.22727,1903.625000,-0.6,29.877273,626,96.55129,97.81904,98.56776,92.34856,96.87456,97.54372,50.1,5182.63
81,04-01-2020,0.789704,19824,7800,83.750000,64.620000,52.771306,11804.00950,1903.367000,-7.1,16.807273,635,98.21212,96.24888,98.10213,95.14948,90.34396,92.28025,49.1,5057.97
82,05-01-2020,0.762035,19441,8908,91.333333,54.148308,66.915249,12179.60526,1975.322632,1.0,28.794762,504,99.01438,95.99664,98.35104,97.71723,93.02601,93.95093,41.5,5239.83


- Removing variables: BCI_China, BCI_Europe, CLI_Europe, Freight_Transport_Index, CLI_China
   1. Because they are least correlated with Target value (StainlessSteelPrice)
   2. By Keeping it : mean error increases and hardly there is a difference in accuracy
   
- Split-Train-Test: 80:20 ~ optimal distribution for training and testing
    1. if 70:30 ~ accuracy drops to 69%, mean error increase to -2.5

In [4]:
predictors = df.drop(columns=['StainlessSteelPrice', 'Date', 'BCI_China', 'BCI_Europe', 'CLI_Europe', 'Freight_Transport_Index', 'CLI_China'], axis=1)
target = df['StainlessSteelPrice']

# Split training and test data or prediction data
X_train, X_cv, y_train, y_cv = train_test_split(predictors, target, test_size=0.20, random_state=1) 

In [5]:
#Hypertuned Model
model = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', min_samples_leaf=3, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=2,
                      oob_score=True, random_state=0, verbose=0)

In [6]:
model.fit(X_train,y_train)

RandomForestRegressor(min_samples_leaf=3, n_estimators=1000, n_jobs=2,
                      oob_score=True, random_state=0)

In [7]:
pred=model.predict(X_cv)

In [8]:
#R2 Score
r2_score(pred,y_cv)

0.8020274080480156

In [9]:
def mean_percentage_error(y_cv, pred):
    y_cv, pred = np.array(y_cv), np.array(pred)
    return np.mean(np.array((y_cv - pred) / y_cv))*100

In [10]:
#Mean error
mean_percentage_error(y_cv,pred)

-1.65946241312756

In [11]:
test1=df.drop(['StainlessSteelPrice', 'Date', 'BCI_China', 'BCI_Europe', 'CLI_Europe', 'Freight_Transport_Index', 'CLI_China'],axis=1)

In [12]:
test1.shape

(84, 13)

In [14]:
pred2=model.predict(test1)

In [15]:
df['StainlessSteelPrice']=pred2

In [16]:
df.head()

,Date,StainlessSteelPrice,Steel_Inventory_M_USD,Steel_Orders_M_USD,IronOre_Global_USD,CoalAustralia_Global_USD,Energy_Index,Nickel_Global_USD,Zinc_Global_USD,Freight_Transport_Index,WTI_Crude_Global_USD,Baltic_Dry_Index,BCI_China,BCI_Europe,BCI_US,CLI_China,CLI_Europe,CLI_US,PMI_US_Manufacturing,Copper_Global_USD
0,07-01-2013,0.804884,20575,10568,127.191304,82.222360,227.022942,13750.31522,1837.619565,-0.4,104.545217,1062,98.80018,99.64265,100.1044,101.1423,99.43615,100.6230,50.9,6906.64
1,08-01-2013,0.780519,20609,10821,137.055000,82.221429,232.234194,14308.26190,1896.392857,0.5,106.546364,1132,99.04874,99.91766,100.3962,101.1938,99.64082,100.6545,55.4,7186.25
2,09-01-2013,0.777600,20761,10774,134.185714,83.295918,234.335594,13801.39286,1846.880952,0.1,106.313810,2003,99.27196,100.18490,100.5363,101.2066,99.84383,100.6646,55.7,7159.27
3,10-01-2013,0.772085,20735,11291,132.572727,85.443634,229.393932,14117.65217,1884.836957,-0.1,100.500435,1504,99.40488,100.36330,100.5771,101.1835,100.02300,100.6666,56.2,7203.02
4,11-01-2013,0.772868,20854,11559,136.323810,88.363265,225.437049,13684.01190,1866.416667,2.1,93.812381,1821,99.36221,100.49830,100.5459,101.1283,100.17150,100.6744,56.4,7070.65


In [17]:
#Display results
result=df[['Date','StainlessSteelPrice']]

In [18]:
result

,Date,StainlessSteelPrice
0,07-01-2013,0.804884
1,08-01-2013,0.780519
2,09-01-2013,0.777600
3,10-01-2013,0.772085
4,11-01-2013,0.772868
...,...,...
79,02-01-2020,0.866548
80,03-01-2020,0.804781
81,04-01-2020,0.728429
82,05-01-2020,0.795335


In [19]:
#Save Final Prediction to csv file
result.to_csv('final_predicted_result.csv')

In [20]:
# Evaluating the Algorithm
print('Mean Absolute Error:', metrics.mean_absolute_error(y_cv, pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_cv, pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_cv, pred)))

Mean Absolute Error: 0.04985051620380731
Mean Squared Error: 0.003662284606343505
Root Mean Squared Error: 0.060516812592398696
